<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
2,95X LEXINGTON PA N,A,Advised,19138762,9954,1,2019-06-26T23:51:39.000,67.0,Proactive Police Visit,Proactive Police Visit,7 - Thomas/Dale(Frogtown),7,2019-06-26T23:51:39.000
3,86X UNIVERSITY AV W,A,Advised,19138761,9954,1,2019-06-26T23:50:07.000,107.0,Proactive Police Visit,Proactive Police Visit,8 - Summit/University,8,2019-06-26T23:50:07.000
4,124X STANTHONY AV,RR,Report Written,19138768,641,1,2019-06-26T23:50:00.000,106.0,Theft,"Theft, From Auto, Under $500",13 - Union Park,13,2019-06-26T23:50:00.000
15,RICE ST & CHARLES,R,Report Written,19138699,600,1,2019-06-26T22:21:28.000,91.0,Theft,"Theft, Except Auto Theft",7 - Thomas/Dale(Frogtown),7,2019-06-26T22:21:28.000


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
2,95x lexington pa n,A,Advised,19138762,9954,1,2019-06-26T23:51:39.000,67.0,Proactive Police Visit,Proactive Police Visit,...,2,0,6,26,177,177,2019-06-26 23:51:39,23,1,0
3,86x university av w,A,Advised,19138761,9954,1,2019-06-26T23:50:07.000,107.0,Proactive Police Visit,Proactive Police Visit,...,2,0,6,26,177,177,2019-06-26 23:50:07,23,1,0
4,124x stanthony av,RR,Report Written,19138768,641,1,2019-06-26T23:50:00.000,106.0,Theft,"Theft, From Auto, Under $500",...,2,0,6,26,177,177,2019-06-26 23:50:00,23,1,0
15,rice st & charles,R,Report Written,19138699,600,1,2019-06-26T22:21:28.000,91.0,Theft,"Theft, Except Auto Theft",...,2,0,6,26,177,177,2019-06-26 22:21:28,22,1,1
18,28x vanburen av,RR,Report Written,19138863,712,1,2019-06-26T22:00:00.000,90.0,Auto Theft,"Motor Vehicle Theft, All Other Vehicles",...,2,0,6,26,177,177,2019-06-26 22:00:00,22,1,0


### Prepare Intersection table <a name="intersection"></a>

In [3]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (5495, 25)


In [4]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns
dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
0,rice_charles,R,Report Written,19138699,600,1,2019-06-26T22:21:28.000,91.0,Theft,"Theft, Except Auto Theft",...,6,26,177,177,2019-06-26 22:21:28,22,1,1,44.957614,-93.105951
1,dale_thomas,A,Advised,19138602,9954,1,2019-06-26T20:38:19.000,88.0,Proactive Police Visit,Proactive Police Visit,...,6,26,177,177,2019-06-26 20:38:19,20,0,1,44.959342,-93.126371
2,dale_university,A,Advised,19138594,9954,1,2019-06-26T20:34:08.000,89.0,Proactive Police Visit,Proactive Police Visit,...,6,26,177,177,2019-06-26 20:34:08,20,0,1,44.955828,-93.126376
3,rice_sherburne,A,Advised,19138447,9954,1,2019-06-26T18:02:23.000,91.0,Proactive Police Visit,Proactive Police Visit,...,6,26,177,177,2019-06-26 18:02:23,18,0,1,44.956848,-93.105964
4,grotto_university,R,Report Written,19138369,600,1,2019-06-26T16:47:58.000,88.0,Theft,"Theft, Except Auto Theft",...,6,26,177,177,2019-06-26 16:47:58,16,0,1,44.955823,-93.131420


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [5]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [7]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'

#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [8]:
fg.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Domestic Assault,Narcotics,Proactive Police Visit,Robbery,Theft,Vandalism,Violent,Advised,Gone on Arrival,Report Written
0,rice_charles,R,Report Written,19138699,600,"44.957614, -93.105951",1,2019-06-26T22:21:28.000,2019-06-26 22:21:28,26,...,0,0,0,0,1,0,0,0,0,1
1,dale_thomas,A,Advised,19138602,9954,"44.959342, -93.126371",1,2019-06-26T20:38:19.000,2019-06-26 20:38:19,26,...,0,0,1,0,0,0,0,1,0,0
2,dale_university,A,Advised,19138594,9954,"44.955828, -93.126376",1,2019-06-26T20:34:08.000,2019-06-26 20:34:08,26,...,0,0,1,0,0,0,0,1,0,0
3,rice_sherburne,A,Advised,19138447,9954,"44.956848, -93.105964",1,2019-06-26T18:02:23.000,2019-06-26 18:02:23,26,...,0,0,1,0,0,0,0,1,0,0
